In [1]:
import dca_v3

In [2]:
dca_v3.carga_bd()
from dca_v3 import *

In [3]:
bd=mx_bd.copy()

In [4]:
display(bd.cuenca.value_counts())

TAMPICO-MISANTLA               1223532
BURGOS                          838977
CUENCAS DEL SURESTE             720643
VERACRUZ                         73609
SABINAS                          10142
CINTURON PLEGADO DE CHIAPAS       1578
Name: cuenca, dtype: int64

In [7]:
def multiple_load(path):
    global df
    files = os.listdir(path)
    files_csv = [f for f in files if f[-3:] == 'csv']
    df = pd.DataFrame()
    for f in files_csv:
        data = pd.read_csv(path+f,
                           header='infer',
                           index_col=9,
                           parse_dates=True,
                           skip_blank_lines=True,
                           infer_datetime_format=True,
                           low_memory=False)

        df = df.append(data,sort=True)

    return df

In [8]:
import os
import pandas as pd

path=os.getcwd()

files=os.listdir()

display(path,files)

df_tipos=pd.read_csv(path + '/output/tampico-misantla/master_df.csv')


'/Users/fffte/Documents/GitHub/projects'

['prototipos',
 '.DS_Store',
 'app',
 'arquitecturas.py',
 'benchmarks_v1.ipynb',
 'productividad',
 'output',
 '__pycache__',
 'README.md',
 '.gitignore',
 '.ipynb_checkpoints',
 'Cheatsheets',
 '.git',
 'dca_v3.py',
 'nb_bayesian.ipynb',
 'app1',
 'nb_tipos.ipynb',
 'workbooks']

In [10]:
arq=df_tipos.trayectoria.value_counts()

In [11]:
display(arq)

DIRECCIONAL      3873
VERTICAL         3536
HORIZONTAL        520
DIRECCIONAL J      34
DIRECCIONAL S       5
DESVIADO            3
MULTILATERAL        1
Name: trayectoria, dtype: int64

In [12]:
keys=arq.index
for x in keys:
    print(x)

DIRECCIONAL
VERTICAL
HORIZONTAL
DIRECCIONAL J
DIRECCIONAL S
DESVIADO
MULTILATERAL


In [29]:
alta=0.80
media=0.50
baja=0.20

x=0

tipos=pd.DataFrame()

for x in keys[:4]:

    serie_tipos=df_tipos[df_tipos.trayectoria == str(x)]

    q_baja=serie_tipos.Qi_hist.quantile(baja)
    q_media=serie_tipos.Qi_hist.quantile(media)
    q_alta=serie_tipos.Qi_hist.quantile(alta)

    criterio1=(serie_tipos['Qi_hist'] <= q_baja)
    tipo1=serie_tipos.loc[criterio1]

    q1_baja=tipo1.Qi_hist.quantile(baja)
    q1_media=tipo1.Qi_hist.quantile(media)
    q1_alta=tipo1.Qi_hist.quantile(alta)

    #d1_baja=tipo1.di_hyp.quantile(baja)
    #d1_media=tipo1.di_hyp.quantile(media)
    #d1_alta_1=tipo1.di_hyp.quantile(alta)

    d1=tipo1.di_hyp.mean()
    b1=tipo1.b.mean()


    ######### POZOS TIPO 2 - Qi MEDIA #########

    criterio2=(serie_tipos['Qi_hist'] > q_baja) & (serie_tipos['Qi_hist'] < q_alta)
    tipo2=serie_tipos.loc[criterio2]

    q2_baja=tipo2.Qi_hist.quantile(baja)
    q2_media=tipo2.Qi_hist.quantile(media)
    q2_alta=tipo2.Qi_hist.quantile(alta)

    #d2_baja=tipo2.di_hyp.quantile(baja)
    #d2_media=tipo2.di_hyp.quantile(media)
    #d2_alta=tipo2.di_hyp.quantile(alta)

    d2=tipo2.di_hyp.mean()
    b2=tipo2.b.mean()


    ######### POZOS TIPO 3 - Qi ALTA #########

    criterio3=(serie_tipos['Qi_hist'] >= q_alta)
    tipo3=serie_tipos.loc[criterio3]

    q3_baja=tipo3.Qi_hist.quantile(baja)
    q3_media=tipo3.Qi_hist.quantile(media)
    q3_alta=tipo3.Qi_hist.quantile(alta)

    #d3_baja=tipo3.di_hyp.quantile(baja)
    #d3_media_3=tipo3.di_hyp.quantile(media)
    #d3_alta=tipo3.di_hyp.quantile(alta)

    d3=tipo3.di_hyp.mean()
    b3=tipo3.b.mean()

    tipo1.loc[:,'tipo']='BAJA_'+str(x)
    tipo2.loc[:,'tipo']='MEDIA_'+str(x)
    tipo3.loc[:,'tipo']='ALTA_'+str(x)

    tipos=tipos.append([tipo1,tipo2,tipo3])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [30]:
display(tipos.tipo.value_counts())

MEDIA_DIRECCIONAL      2323
MEDIA_VERTICAL         2119
BAJA_DIRECCIONAL        775
ALTA_DIRECCIONAL        775
ALTA_VERTICAL           709
BAJA_VERTICAL           708
MEDIA_HORIZONTAL        312
ALTA_HORIZONTAL         104
BAJA_HORIZONTAL         104
MEDIA_DIRECCIONAL J      20
ALTA_DIRECCIONAL J        7
BAJA_DIRECCIONAL J        7
Name: tipo, dtype: int64

In [31]:
df=tipos.groupby(by='tipo').mean()

In [32]:
display(df)

,Qi_hist,mes_max,profundidad_total,Np,Gp,Cp,Wp,acum_BOE,RGA,corte_agua,...,di_exp_condensado,RSS_exponencial,RSS_hiperbolica,RSS_harmonica,RSS_gas_exponencial,RSS_gas_hiperbolica,RSS_gas_harmonica,RSS_condensado_exponencial,RSS_condensado_hiperbolica,RSS_condensado_harmonica
tipo,,,,,,,,,,,,,,,,,,,,,
ALTA_DIRECCIONAL,0.441270,186.601290,2260.052169,0.528106,0.928243,0.0,0.243944,0.682813,73427.873562,0.710238,...,5.0,4.259688,2.780270,2.966037,45.012762,28.767754,29.745734,45.012762,28.767754,29.745734
ALTA_DIRECCIONAL J,0.283857,15.428571,2019.157143,0.055719,0.041299,0.0,0.007937,0.062602,902.516226,0.138386,...,5.0,0.043843,0.041978,0.042834,0.033546,0.032884,0.032884,0.033546,0.032884,0.032884
ALTA_HORIZONTAL,0.510799,97.230769,2532.852212,0.383628,0.445968,0.0,0.207513,0.457956,2551.911866,1.060306,...,5.0,0.801944,0.778642,1.055951,3.691573,3.674305,3.772010,3.691573,3.674305,3.772010
ALTA_VERTICAL,2.799869,486.812412,2542.607669,2.308606,1.647947,0.0,0.702241,2.583264,1886.640371,1.055486,...,5.0,41.535686,37.909439,42.067471,44.715136,42.371649,44.192319,44.715136,42.371649,44.192319
BAJA_DIRECCIONAL,0.017365,127.301935,2029.919496,0.032041,0.071810,0.0,0.016587,0.044009,29796.852041,0.834951,...,5.0,0.015734,0.014730,0.014749,0.195403,0.188283,0.189315,0.195403,0.188283,0.189315
BAJA_DIRECCIONAL J,0.018682,3.571429,1761.666667,0.001566,0.003217,0.0,0.000857,0.002103,9200.779374,1.882622,...,5.0,0.000034,0.000021,0.000021,0.000215,0.000200,0.000200,0.000215,0.000200,0.000200
BAJA_HORIZONTAL,0.024129,44.019231,1019.285288,0.028972,0.003884,0.0,0.008656,0.029619,646.318520,0.884349,...,5.0,0.024845,0.023686,0.023719,0.003161,0.003150,0.003152,0.003161,0.003150,0.003152
BAJA_VERTICAL,0.009380,219.693503,1255.304318,0.061492,0.113739,0.0,0.057687,0.080448,128506.738460,1.621903,...,5.0,0.037276,0.036341,0.036354,0.932878,0.927786,0.939692,0.932878,0.927786,0.939692
MEDIA_DIRECCIONAL,0.073106,133.958244,2111.857107,0.068541,0.144810,0.0,0.034127,0.092676,3213.731601,0.571369,...,5.0,0.088854,0.077034,0.077996,1.570181,1.509661,1.513344,1.570181,1.509661,1.513344


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None


In [ ]:
# read the data
df = mx_bd

# shape and data types of the data
print(df.shape)
print(df.dtypes)

# select numeric columns
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)

# select non numeric columns
df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print(non_numeric_cols)

# Dealing with missing data

## Technique 1: Heatmap

In [ ]:
cols = df.columns[:30] # first 30 columns
colours = ['#000099', '#ffff00'] # specify the colours - yellow is missing. blue is not missing.
sns.heatmap(df[cols].isnull(), cmap=sns.color_palette(colours))

## Technique 2: Percentage of data missing

In [ ]:
# if it's a larger dataset and the visualization takes too long can do this.
# % of missing.
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

## Technique 3: Histogram

In [ ]:
# first create missing indicator for features with missing data
for col in df.columns:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  
        print('created missing indicator for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing


# then based on the indicator, plot the histogram of missing values
ismissing_cols = [col for col in df.columns if 'ismissing' in col]
df['num_missing'] = df[ismissing_cols].sum(axis=1)

df['num_missing'].value_counts().reset_index().sort_values(by='index').plot.bar(x='index', y='num_missing')

In [ ]:
dca_v3.inputs()

In [ ]:
from dca_v3 import *

In [ ]:
display(pozos.pozo.value_counts())